<a href="https://colab.research.google.com/github/onlygoodman/dacon_shopping_reviewscore_predict/blob/main/project_shop_013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

001 - https://wikidocs.net/94600 네이버 쇼핑 리뷰 데이터 분석을 참조하여 0,1 긍부정 이진 분류를 통해 확률을 0~5로 확장하여 예측


002 - 001 코드에서 target 그대로 활용, 다중분류로 예측 시도

003 - 002 코드에 dense layer 64, 32, 16, 8 사이에 각각 dropdout 0.2~0.5 적용하여 추가학습 epoch 25로 변경 early stop 인내값 8로 변경

004 - 003에서 epoch, batch-size 등 hyper parameter search에 유의미한 결과를 얻지 못하여 hyperband를 활용하여 하이퍼파라미터 튜닝 시도

005 - 004에서 얻은 하이퍼파라미터 튜닝 결과 대입하여 val_acc 측정 시도 **최고점수

006 - 005 모델에서 배치사이즈 16, denselayer 16짜리 하나 추가

007 - kobert 활용하여 결과도출

# colab에서 kobert 동작환경 구축

In [ ]:
# import tensorflow as tf
# import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 31.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=c60c940d346b14a597f4c35b8cf149ec49ac3b047d1d0c484e2d708b182016eb
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 33.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ppmfyl6y/kobert-tokenizer_ab0bed96127a45f6b9b0d82af6e539e6
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ppmfyl6y/kobert-tokenizer_ab0bed96127a45f6b9b0d82af6e539e6
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=5497bc8916029e7d2b229b74701298ab388cb8a5b319d3d1e41ebb6133d00824
  Stored in directory: /tmp/pip-ephem-wheel-cache-kqhb1dnf/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7n6h8opa
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-7n6h8opa
     |████████████████████████████████| 132 kB 19.4 MB/s 
     |████████████████████████████████| 4.5 MB 55.9 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 9.0 MB 64.4 MB/s 
     |████████████████████████████████| 127 kB 73.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=db74a32c9146a161e71045e43031090416593ab79d23aa9d5a1bb42f5a49f344
  Stored in directory: /tmp/pip-ephem-wheel-cache-1s2em4bp/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling url

In [ ]:
import os
from zipfile import ZipFile
import torch
from transformers import BertModel
import gluonnlp as nlp

from kobert import download, get_tokenizer

In [ ]:
def get_pytorch_kobert_model(ctx="cpu", cachedir=".cache"):
    def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

    pytorch_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert_v1.zip",
        "chksum": "411b242919",  # 411b2429199bc04558576acdcac6d498
    }

    # download model
    model_info = pytorch_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    zipf = ZipFile(os.path.expanduser(model_path))
    zipf.extractall(path=cachedir_full)
    model_path = os.path.join(os.path.expanduser(cachedir), "kobert_from_pretrained")
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(model_path, vocab_path, ctx)


if __name__ == "__main__":
    import torch
    from kobert import get_pytorch_kobert_model

    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    model, vocab = get_pytorch_kobert_model()
    sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)
    print(pooled_output.shape)
    print(vocab)
    print(sequence_output[0])

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
torch.Size([2, 768])
Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward0>)


In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from kobert import get_pytorch_kobert_model
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab  = get_pytorch_kobert_model()

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# 데이터 준비

In [ ]:
# Train dataset은 Label이 존재하지 않음
train_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit = pd.read_csv("/content/drive/MyDrive/shopping_dataset/sample_submission.csv")

In [ ]:
train_data

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5
...,...,...,...
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2


In [ ]:
def changetarget(data):
  if data ==1:
    return 0
  elif data ==2:
    return 1

  elif data ==4:
    return 2
  elif data ==5:
    return 3

train_data['target']=train_data['target'].apply(lambda x : changetarget(x))
train_data

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,1
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,0
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,1
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,1
4,4,튼튼하고 손목을 잘 받쳐주네요~,3
...,...,...,...
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,3
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,1
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,2
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,1


In [ ]:
data_list = []
for ques, label in zip(train_data['reviews'], train_data['target'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[:5]

[['조아요 처음구입 싸게햇어요', '1'],
 ['생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 아깝네요', '0'],
 ['디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져있네요 ㅠ 귀찮아서 그냥쓰려구요 배송기사님은 친절하셨어요',
  '1'],
 ['기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘플 써보고 마음에 들어 구매해 사용중 같은 제품을 팔지 않아 할수 없이 플러스를 구매했는데요, 감고나도 머리가 개운하지 않고 간지럽네요. 저랑은 잘 안맞는듯해요.',
  '1'],
 ['튼튼하고 손목을 잘 받쳐주네요~', '3']]

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

# tokenizer 동작 확인

In [ ]:
!pip install --no-cache-dir transformers sentencepiece 

from transformers import AutoTokenizer, AutoModelForMaskedLM

# kcbert의 tokenizer와 모델을 불러옴.
kcbert_tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
kcbert = AutoModelForMaskedLM.from_pretrained("beomi/kcbert-base")

result = kcbert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
print(kcbert_tokenizer.vocab['대선'])
print([kcbert_tokenizer.encode(token) for token in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['너는', '내년', '대선', '때', '투표', '##할', '수', '있어', '?']
9311
[[2, 9039, 3], [2, 8946, 3], [2, 9311, 3], [2, 1003, 3], [2, 8701, 3], [2, 7, 7, 3358, 3], [2, 1931, 3], [2, 8749, 3], [2, 32, 3]]


In [ ]:
!pip install --no-cache-dir transformers sentencepiece

from transformers import AutoModel, AutoTokenizer

kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
kobert = AutoModel.from_pretrained("skt/kobert-base-v1")

result = kobert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
kobert_vocab = kobert_tokenizer.get_vocab()
print(kobert_vocab.get('▁대선'))
print([kobert_tokenizer.encode(token) for token in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

['▁너', '는', '▁내년', '▁대선', '▁때', '▁투표', '할', '▁수', '▁있어', '?']
1654
[[1457, 3, 2], [517, 5760, 3, 2], [1437, 3, 2], [1654, 3, 2], [1844, 3, 2], [4772, 3, 2], [4977, 3, 2], [2872, 3, 2], [3868, 3, 2], [633, 3, 2]]


# model 파라미터 지정 및 학습데이터 지정

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.05
num_epochs = 3  
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, shuffle=True, random_state=42)

In [ ]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=10)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=10)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 모델 학습 및 예측

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np
# from typing import Optional

# def label_to_one_hot_label(
#     labels: torch.Tensor,
#     num_classes: int,
#     device: Optional[torch.device] = None,
#     dtype: Optional[torch.dtype] = None,
#     eps: float = 1e-6,
#     ignore_index=255,
# ) -> torch.Tensor:
#     r"""Convert an integer label x-D tensor to a one-hot (x+1)-D tensor.

#     Args:
#         labels: tensor with labels of shape :math:`(N, *)`, where N is batch size.
#           Each value is an integer representing correct classification.
#         num_classes: number of classes in labels.
#         device: the desired device of returned tensor.
#         dtype: the desired data type of returned tensor.

#     Returns:
#         the labels in one hot tensor of shape :math:`(N, C, *)`,

#     Examples:
#         >>> labels = torch.LongTensor([
#                 [[0, 1], 
#                 [2, 0]]
#             ])
#         >>> one_hot(labels, num_classes=3)
#         tensor([[[[1.0000e+00, 1.0000e-06],
#                   [1.0000e-06, 1.0000e+00]],
        
#                  [[1.0000e-06, 1.0000e+00],
#                   [1.0000e-06, 1.0000e-06]],
        
#                  [[1.0000e-06, 1.0000e-06],
#                   [1.0000e+00, 1.0000e-06]]]])

#     """
#     shape = labels.shape
#     # one hot : (B, C=ignore_index+1, H, W)
#     one_hot = torch.zeros((shape[0], ignore_index+1) + shape[1:], device=device, dtype=dtype)
    
#     # labels : (B, H, W)
#     # labels.unsqueeze(1) : (B, C=1, H, W)
#     # one_hot : (B, C=ignore_index+1, H, W)
#     one_hot = one_hot.scatter_(1, labels.unsqueeze(1), 1.0) + eps
    
#     # ret : (B, C=num_classes, H, W)
#     ret = torch.split(one_hot, [num_classes, ignore_index+1-num_classes], dim=1)[0]
    
#     return ret


# # https://github.com/zhezh/focalloss/blob/master/focalloss.py
# def focal_loss(input, target, alpha, gamma, reduction, eps, ignore_index):
    
#     r"""Criterion that computes Focal loss.

#     According to :cite:`lin2018focal`, the Focal loss is computed as follows:

#     .. math::

#         \text{FL}(p_t) = -\alpha_t (1 - p_t)^{\gamma} \, \text{log}(p_t)

#     Where:
#        - :math:`p_t` is the model's estimated probability for each class.

#     Args:
#         input: logits tensor with shape :math:`(N, C, *)` where C = number of classes.
#         target: labels tensor with shape :math:`(N, *)` where each value is :math:`0 ≤ targets[i] ≤ C−1`.
#         alpha: Weighting factor :math:`\alpha \in [0, 1]`.
#         gamma: Focusing parameter :math:`\gamma >= 0`.
#         reduction: Specifies the reduction to apply to the
#           output: ``'none'`` | ``'mean'`` | ``'sum'``. ``'none'``: no reduction
#           will be applied, ``'mean'``: the sum of the output will be divided by
#           the number of elements in the output, ``'sum'``: the output will be
#           summed.
#         eps: Scalar to enforce numerical stabiliy.

#     Return:
#         the computed loss.

#     Example:
#         >>> N = 5  # num_classes
#         >>> input = torch.randn(1, N, 3, 5, requires_grad=True)
#         >>> target = torch.empty(1, 3, 5, dtype=torch.long).random_(N)
#         >>> output = focal_loss(input, target, alpha=0.5, gamma=2.0, reduction='mean')
#         >>> output.backward()
#     """
#     if not isinstance(input, torch.Tensor):
#         raise TypeError(f"Input type is not a torch.Tensor. Got {type(input)}")

#     if not len(input.shape) >= 2:
#         raise ValueError(f"Invalid input shape, we expect BxCx*. Got: {input.shape}")

#     if input.size(0) != target.size(0):
#         raise ValueError(f'Expected input batch_size ({input.size(0)}) to match target batch_size ({target.size(0)}).')

#     # input : (B, C, H, W)
#     n = input.size(0) # B
    
#     # out_sie : (B, H, W)
#     out_size = (n,) + input.size()[2:]
    
#     # input : (B, C, H, W)
#     # target : (B, H, W)
#     if target.size()[1:] != input.size()[2:]:
#         raise ValueError(f'Expected target size {out_size}, got {target.size()}')

#     if not input.device == target.device:
#         raise ValueError(f"input and target must be in the same device. Got: {input.device} and {target.device}")
    
#     if isinstance(alpha, float):
#         pass
#     elif isinstance(alpha, np.ndarray):
#         alpha = torch.from_numpy(alpha)
#         # alpha : (B, C, H, W)
#         alpha = alpha.view(-1, len(alpha), 1, 1).expand_as(input)
#     elif isinstance(alpha, torch.Tensor):
#         # alpha : (B, C, H, W)
#         alpha = alpha.view(-1, len(alpha), 1, 1).expand_as(input)       
        

#     # compute softmax over the classes axis
#     # input_soft : (B, C, H, W)
#     input_soft = F.softmax(input, dim=1) + eps
    
#     # create the labels one hot tensor
#     # target_one_hot : (B, C, H, W)
#     target_one_hot = label_to_one_hot_label(target.long(), num_classes=input.shape[1], device=input.device, dtype=input.dtype, ignore_index=ignore_index)

#     # compute the actual focal loss
#     weight = torch.pow(1.0 - input_soft, gamma)
    
#     # alpha, weight, input_soft : (B, C, H, W)
#     # focal : (B, C, H, W)
#     focal = -alpha * weight * torch.log(input_soft)
    
#     # loss_tmp : (B, H, W)
#     loss_tmp = torch.sum(target_one_hot * focal, dim=1)

#     if reduction == 'none':
#         # loss : (B, H, W)
#         loss = loss_tmp
#     elif reduction == 'mean':
#         # loss : scalar
#         loss = torch.mean(loss_tmp)
#     elif reduction == 'sum':
#         # loss : scalar
#         loss = torch.sum(loss_tmp)
#     else:
#         raise NotImplementedError(f"Invalid reduction mode: {reduction}")
#     return loss


# class FocalLoss(nn.Module):
#     r"""Criterion that computes Focal loss.

#     According to :cite:`lin2018focal`, the Focal loss is computed as follows:

#     .. math:

#         FL(p_t) = -alpha_t(1 - p_t)^{gamma}, log(p_t)

#     Where:
#        - :math:`p_t` is the model's estimated probability for each class.

#     Args:
#         alpha: Weighting factor :math:`\alpha \in [0, 1]`.
#         gamma: Focusing parameter :math:`\gamma >= 0`.
#         reduction: Specifies the reduction to apply to the
#           output: ``'none'`` | ``'mean'`` | ``'sum'``. ``'none'``: no reduction
#           will be applied, ``'mean'``: the sum of the output will be divided by
#           the number of elements in the output, ``'sum'``: the output will be
#           summed.
#         eps: Scalar to enforce numerical stabiliy.

#     Shape:
#         - Input: :math:`(N, C, *)` where C = number of classes.
#         - Target: :math:`(N, *)` where each value is
#           :math:`0 ≤ targets[i] ≤ C−1`.

#     Example:
#         >>> N = 5  # num_classes
#         >>> kwargs = {"alpha": 0.5, "gamma": 2.0, "reduction": 'mean'}
#         >>> criterion = FocalLoss(**kwargs)
#         >>> input = torch.randn(1, N, 3, 5, requires_grad=True)
#         >>> target = torch.empty(1, 3, 5, dtype=torch.long).random_(N)
#         >>> output = criterion(input, target)
#         >>> output.backward()
#     """

#     def __init__(self, alpha, gamma = 2.0, reduction = 'mean', eps = 1e-8, ignore_index=30):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.reduction = reduction
#         self.eps = eps
#         self.ignore_index = ignore_index

#     def forward(self, input, target):
#         return focal_loss(input, target, self.alpha, self.gamma, self.reduction, self.eps, self.ignore_index)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [ ]:
# class WeightedFocalLoss(nn.Module):
#     def __init__(self, alpha=.25, gamma=2):
#         super(WeightedFocalLoss, self).__init__()
#         self.alpha = torch.tensor([alpha, 1-alpha]).cuda()
#         self.gamma = gamma

#     def forward(self, inputs, targets):
#         BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
#         targets = targets.type(torch.long)
#         at = self.alpha.gather(0, targets.data.view(-1))
#         pt = torch.exp(-BCE_loss)
#         F_loss = at*(1-pt)**self.gamma * BCE_loss
#         return F_loss.mean()

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.2).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = FocalLoss() # 다중분류를 위한 대표적인 loss func // 현재는 crossentrophy 문제를 개선한 focalloss 사용

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/352 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.9254152178764343 train acc 0.140625
epoch 1 batch id 201 loss 0.23700957000255585 train acc 0.6469993781094527
epoch 1 train acc 0.6728910195707071


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.685546875


  0%|          | 0/352 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.18089134991168976 train acc 0.6875
epoch 3 batch id 201 loss 0.15326420962810516 train acc 0.7533426616915423
epoch 3 train acc 0.7602292455808081


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 3 test acc 0.683984375


# test데이터의 예측값 도출

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0, pin_memory = True)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(1)
            elif np.argmax(logits) == 1:
                test_eval.append(2)
            elif np.argmax(logits) == 2:
                test_eval.append(4)
            elif np.argmax(logits) == 3:
                test_eval.append(5)

        # print(">> 입력하신 내용의 평점은 {}점 입니다.".format(test_eval[0]))
        return test_eval[0]

In [ ]:
sentest= ["아진짜 좋아", "이거 대박", "정말 좋긴한데.. 뭔가 아쉬워요", "이거 진짜 진짜 너무 너무 별로다 ... 조상님이 울고가겠어요"]

for sen1 in sentest:
  print(sen1)
  print(predict(sen1))
  print()

아진짜 좋아
2

이거 대박
5

정말 좋긴한데.. 뭔가 아쉬워요
2

이거 진짜 진짜 너무 너무 별로다 ... 조상님이 울고가겠어요
1



In [ ]:
test_sen = test_data.reviews

pred = []

for sentence in tqdm_notebook(test_sen):
  pred.append(predict(sentence))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25000 [00:00<?, ?it/s]

In [ ]:
print(len(pred))
len(test_sen)

25000


25000

In [ ]:
projectname = "shop"
model = "KOBERT-hyperparamsCHANGE_addFocalLoss"
testnum = "013"

submit["target"] = pred
submit.head() 
submit.to_csv("sub_{}_{}_model_{}.csv".format(testnum, projectname, model),index=False)

In [ ]:
# test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit.to_csv("/content/drive/MyDrive/shopping_dataset/sub_{}_{}_model_{}.csv".format(testnum, projectname, model),index=False)
